In [8]:
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from seleniumwire import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

# Set up the WebDriver
chrome_service = ChromeService(ChromeDriverManager().install())
# Configure Chrome options
options = webdriver.ChromeOptions()
# options.add_argument("--auto-open-devtools-for-tabs")  # Open DevTools automatically

# Initialize the WebDriver with options
driver = webdriver.Chrome(service=chrome_service, options=options)

driver.get('https://www.naturalreaders.com/online')

In [18]:
# JavaScript code to intercept blob URLs
intercept_blob_urls_js = """
if (!window.__capturedBlobUrls) {        
    window.__capturedBlobUrls = [];        
    const originalCreateObjectURL = URL.createObjectURL;        
    URL.createObjectURL = function(blob) {            
        const url = originalCreateObjectURL(blob);                
        window.__capturedBlobUrls.push(url);                
        console.log('Blob URL captured:', url);                
        return url;            
    };        
    console.log('Blob URL capture is now active.');
}
"""

# Execute the script
driver.execute_script(intercept_blob_urls_js)
captured_blob_urls = driver.execute_script("return window.__capturedBlobUrls;")
captured_blob_urls

['blob:https://www.naturalreaders.com/3fe1ee4a-fcf9-4005-8b4c-bd5ed7ed7616',
 'blob:https://www.naturalreaders.com/4b681a79-b20d-412d-8764-f8bfe74d1bc8',
 'blob:https://www.naturalreaders.com/71f1bd99-5d92-4758-aa1b-d8ffd5698310']

In [29]:
import base64
import os
import requests

folder_path = './Audio'
file_name_base = 'Blue-Q1'
os.makedirs(folder_path, exist_ok=True)


for index, blob_url in enumerate(captured_blob_urls):
    # Get the blob content using JavaScript
    script = f"""
    return fetch('{blob_url}')
        .then(response => response.blob())
        .then(blob => new Promise((resolve, reject) => {{
            const reader = new FileReader();
            reader.onloadend = () => resolve(reader.result);
            reader.onerror = reject;
            reader.readAsDataURL(blob);
        }}));
    """
    blob_data_url = driver.execute_script(script)
    blob_base64 = blob_data_url.split(',')[1]

    # Decode the base64 encoded data
    blob_content = base64.b64decode(blob_base64)
    
    file_path = os.path.join(folder_path, f"{file_name_base}-{index + 1}.mp3")
    
    # Write content to the file
    with open(file_path, 'wb') as f:
        f.write(blob_content)
    
    print(f"Downloaded and saved: {file_path}")


Downloaded and saved: ./Audio/Blue-Q1-1.mp3
Downloaded and saved: ./Audio/Blue-Q1-2.mp3
Downloaded and saved: ./Audio/Blue-Q1-3.mp3
